In [1]:
from thinc.api import IvyWrapper, Adam, CategoricalCrossentropy

2023-09-16 07:16:51.442856: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import ivy

In [3]:
import ml_datasets
from tqdm import tqdm

In [4]:
(train_X, train_Y), (dev_X, dev_Y) = ml_datasets.mnist()
print(f"Training size={len(train_X)}, dev size={len(dev_X)}")

Training size=54000, dev size=10000


In [5]:
def train_model(data, model, optimizer, n_iter, batch_size):
    (train_X, train_Y), (dev_X, dev_Y) = data
    indices = model.ops.xp.arange(train_X.shape[0], dtype="i")
    for i in range(n_iter):
        batches = model.ops.multibatch(batch_size, train_X, train_Y, shuffle=True)
        for X, Y in tqdm(batches, leave=False):
            Yh, backprop = model.begin_update(X)
            backprop(Yh - Y)
            model.finish_update(optimizer)
        # Evaluate and print progress
        correct = 0
        total = 0
        for X, Y in model.ops.multibatch(batch_size, dev_X, dev_Y):
            Yh = model.predict(X)
            correct += (Yh.argmax(axis=1) == Y.argmax(axis=1)).sum()
            total += Yh.shape[0]
        score = correct / total
        print(f" {i} {float(score):.3f}")

In [6]:
width = 32
nO = 10
nI = 784
dropout = 0.2

In [7]:
class Regressor(ivy.Module):
    def __init__(self, width, nO, nI, dropout):
        self.nO = nO
        self.nI = nI
        self.dropout = dropout
        self.width = width
        super().__init__()

    def _build(self, *args, **kwargs):
        self.dropout1 = ivy.Dropout(self.dropout)
        self.dropout2 = ivy.Dropout(self.dropout)
        self.fc1 = ivy.Linear(self.nI, self.width)
        self.fc2 = ivy.Linear(self.width, self.nO)

    def _forward(self, x):
        x = ivy.relu(x)
        x = self.dropout1(x)
        x = self.fc1(x)
        x = ivy.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = ivy.log_softmax(x, axis=1)
        return output

In [8]:
model = IvyWrapper(
    Regressor(width, nO, nI, dropout),
    )

In [9]:
data = ml_datasets.mnist()
optimizer = Adam(0.001)
model.initialize(X=train_X[:5], Y=train_Y[:5])
train_model(data, model, optimizer, n_iter=10, batch_size=128)

 0 0.489


 1 0.542


 2 0.609


 3 0.678


 4 0.674


 5 0.667


 6 0.683


 7 0.685


 8 0.674


 9 0.672
